In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

In [2]:
file_1="Dataset/B0005_dis_dataset.csv"
dataset=pd.read_csv(file_1)

In [3]:
C_max=2

#renaming columns for simplicity
dataset.columns=['cycle',"capacity",'voltage', 'current','temperature', 'current_load', 'voltage_load',"time"]

#removing ideling / starting and ending current level data  

df=dataset[dataset.current_load<-0.0006].reset_index(drop=True)

# converting time seconds->hour

df["time"]=df["time"]/3600

In [4]:
#calculate soc for each cycle
cycle_count=df["cycle"].unique()
DOD_list=[]

In [5]:

# Update SOC for each reading within each cycle using Coulomb counting
for cycle in cycle_count:
    cycle_data = df[df['cycle'] == cycle]
    
    cumulative_DOD = [2.0]
    
    for i in range(1, len(cycle_data)):
        
        delta_time = cycle_data['time'].iloc[i-1] - cycle_data['time'].iloc[i]
        
        delta_charge = cycle_data['current'].iloc[i] * delta_time
        
        cumulative_DOD.append(cumulative_DOD[-1] -delta_charge)
        
    DOD_list.extend(cumulative_DOD)


In [6]:
#DOD is the opposite if the soc 
DOD = [(q / C_max) for q in DOD_list]
df["DOD"]=DOD

In [7]:
df

cycle  capacity   voltage   current  temperature  current_load  \
0         1  1.856487  3.974871 -2.012528    24.389085       -1.9982   
1         1  1.856487  3.951717 -2.013979    24.544752       -1.9982   
2         1  1.856487  3.934352 -2.011144    24.731385       -1.9982   
3         1  1.856487  3.920058 -2.013007    24.909816       -1.9982   
4         1  1.856487  3.907904 -2.014400    25.105884       -1.9982   
...     ...       ...       ...       ...          ...           ...   
5426     43  1.767617  2.833512 -2.013403    38.178902       -1.9982   
5427     43  1.767617  2.684431 -2.013194    38.464464       -1.9984   
5428     43  1.767617  3.064858 -0.000809    38.503541       -0.0008   
5429     43  1.767617  3.227714  0.000523    38.152458       -0.0008   
5430     43  1.767617  3.301705 -0.003164    37.401689       -0.0008   

      voltage_load      time       DOD  
0            3.062  0.009918  1.000000  
1            3.030  0.014939  0.994943  
2            3.011  0.019978  0.989876  
3            2.991  0.025026  0.984795  
4            2.977  0.030078  0.979707  
...            ...       ...       ...  
5426         1.964  0.880091  0.124314  
5427         1.846  0.885551  0.118818  
5428         0.000  0.891059  0.118815  
5429         0.000  0.902092  0.118818  
5430         0.000  0.913203  0.118801  

[5431 rows x 9 columns]

In [8]:
cor = df.corr()
plt.figure(figsize=(10,6))  #select value out of range(0.05,-0.05)
sns.heatmap(cor[["voltage_load"]],annot=True,cmap='coolwarm')

<Axes: >

In [9]:
#important from this is temprature,DOD,time

In [10]:
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y,)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [11]:
mi_scores = make_mi_scores(df,df[["voltage_load"]])

mi_scores

/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


voltage_load    6.358460
voltage         3.873177
time            3.573053
DOD             3.412504
temperature     2.333272
current         0.027433
current_load    0.016730
cycle           0.000000
capacity        0.000000
Name: MI Scores, dtype: float64

In [12]:
def plot_mi_scores(scores):
    plt.figure(dpi=200, figsize=(12, 5))
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    for index, value in enumerate(scores):
        plt.text(value, index,str(round(value,3)))
plot_mi_scores(mi_scores)

In [ ]:
#temp,DOD,temprature,